In [ ]:
from sqlalchemy import create_engine
import pandas as pd

db_con = create_engine('postgresql://postgres:argmax@pg:5432/postgres')


In [15]:


# Define your SQL query
sql_query = 'SELECT * FROM INFORMATION_SCHEMA.TABLES;'

df = pd.read_sql(sql_query, con=db_con)

print(df)


    table_catalog        table_schema              table_name  table_type  \
0        postgres          pg_catalog            pg_statistic  BASE TABLE   
1        postgres          pg_catalog                 pg_type  BASE TABLE   
2        postgres          pg_catalog        pg_foreign_table  BASE TABLE   
3        postgres          pg_catalog               pg_authid  BASE TABLE   
4        postgres          pg_catalog               pg_shadow        VIEW   
..            ...                 ...                     ...         ...   
204      postgres  information_schema          foreign_tables        VIEW   
205      postgres  information_schema  foreign_server_options        VIEW   
206      postgres  information_schema       _pg_user_mappings        VIEW   
207      postgres  information_schema    user_mapping_options        VIEW   
208      postgres  information_schema           user_mappings        VIEW   

    self_referencing_column_name reference_generation  \
0                 